In [1]:
import pandas as pd
import datetime
import numpy as np

s_metar = "C:\\Users\\Matvey Solovyov\\OneDrive\\Documents\\UWA\\2021_S1\\GinginWx\\gingin-weather\\gingin_metar_prcpINT.csv"
s_model = "C:\\Users\\Matvey Solovyov\\OneDrive\\Documents\\UWA\\2021_S1\\GinginWx\\gingin-weather\\modelData.csv"

df_metar = pd.read_csv(s_metar)
df_model = pd.read_csv(s_model)

for df in [df_model,df_metar]:
    df['date'] = pd.to_datetime(df['date'])

In [25]:
df_metar_comp = df_metar.copy()
df_model_comp = df_model.copy()

df_metar_comp = df_metar_comp[ df_metar_comp.date.gt( datetime.datetime( year = 2015, month = 1, day = 1 ) ) &
         df_metar_comp.date.lt( datetime.datetime( year = 2019, month = 2, day = 1)) ]
df_model_comp = df_model_comp[ df_model_comp.date.gt( datetime.datetime( year = 2015, month = 1, day = 1 ) ) &
         df_model_comp.date.lt( datetime.datetime( year = 2019, month = 2, day = 1)) ]

In [30]:
df_metar_comp['delta'] = df_metar_comp.date - df_metar_comp.date.shift(1)
df_model_comp['delta'] = df_model_comp.date - df_model_comp.date.shift(1)

print( df_model_comp[ df_model_comp.pluie.gt(0) ].delta.sum() / df_model_comp.delta.sum() )
print( df_metar_comp[ df_metar_comp.prcp.gt(0) ].delta.sum() / df_metar_comp.delta.sum() )

0.055219659135472075
0.04021025509689336


In [31]:
(0.055219659135472075 - 0.04021025509689336) / 0.055219659135472075

0.27181268905980904

In [2]:
print("Number of time gaps longer than an hour: ", len(df_model[ df_model['date'].diff() > datetime.timedelta( hours = 1 ) ]) )

# 296 times the gap is longer than an hour. Maybe split these?

# for reference
print("mean pluie: ", df_model['pluie'].mean())

Number of time gaps longer than an hour:  296
mean pluie:  0.0342974649304519


In [162]:
df_model = df_model.set_index('date')
df_model['date'] = df_model.index
df_model_hourly = df_model.resample('H').mean()

df_model_hourly = df_model_hourly.dropna()

# For all else best to have mean, then change pluie to binary if > 0

df_model_hourly['pluie'] = (df_model_hourly['pluie'] > 0).astype(int)

df_model_hourly['pluie'].mean()


0.077162913340935

In [163]:
print("Number of time gaps longer than an hour: ", len(df_metar[ df_metar['date'].diff() > datetime.timedelta( hours = 1 ) ]) )

# 296 times the gap is longer than an hour. Maybe split these?

# for reference
print("mean precip: ", df_metar['prcp'].mean())

Number of time gaps longer than an hour:  191
mean precip:  0.046081347510456186


In [17]:
df_model['date'].iloc[ df_model['pluie'].ne(0).idxmax() ]

## First non-zero pluie happens only in 114,441 rows (19 Feb 2015)
## Assume the rain sensor operational from 114,440 (unless we can find out when it was installed exactly)
## Hence we will only work with data from this point

Timestamp('2015-02-19 21:23:33')

In [180]:
df_metar = df_metar.set_index('date')
df_metar['date'] = df_metar.index
df_metar_hourly = df_metar.resample('H').sum()

df_metar_hourly = df_metar_hourly.dropna()

df_metar_hourly['prcp'].mean()

0.06976603747541323

In [181]:
# 2020 11 10 end
# 2013 11 26 start

df_joined_hourly = df_model_hourly.join( df_metar_hourly )
df_joined_hourly = df_joined_hourly.dropna()

In [182]:
df_joined_hourly['prcp_bool'] = (df_joined_hourly['prcp'] > 0).astype(int)

In [191]:
bad1 = 2
i = 0

while bad1 != 1:
    bad1 = len(df_joined_hourly[ (df_joined_hourly['pluie'] != df_joined_hourly['prcp_bool']) & (df_joined_hourly['prcp'] > i)  ])
    print( bad1, " rows showing no pluie with prcp greater than ", i ,"mm")
    i += 1
    
bad2 = len( df_joined_hourly[ (df_joined_hourly['pluie'] == 1 ) & (df_joined_hourly['prcp'] == 0) ] )
print("\n{0} rows showing pluie with no prcp\n".format(bad2))

good = len( df_joined_hourly[ df_joined_hourly['pluie'] == df_joined_hourly['prcp_bool']  ])

print(good,"rows have matching pluie and prcp columns ({0}%)".format( round( (good/len(df_joined_hourly))*100 , 4) ))

2271  rows showing no pluie with prcp greater than  0 mm
508  rows showing no pluie with prcp greater than  1 mm
253  rows showing no pluie with prcp greater than  2 mm
151  rows showing no pluie with prcp greater than  3 mm
100  rows showing no pluie with prcp greater than  4 mm
66  rows showing no pluie with prcp greater than  5 mm
48  rows showing no pluie with prcp greater than  6 mm
30  rows showing no pluie with prcp greater than  7 mm
22  rows showing no pluie with prcp greater than  8 mm
15  rows showing no pluie with prcp greater than  9 mm
12  rows showing no pluie with prcp greater than  10 mm
9  rows showing no pluie with prcp greater than  11 mm
7  rows showing no pluie with prcp greater than  12 mm
6  rows showing no pluie with prcp greater than  13 mm
5  rows showing no pluie with prcp greater than  14 mm
4  rows showing no pluie with prcp greater than  15 mm
3  rows showing no pluie with prcp greater than  16 mm
3  rows showing no pluie with prcp greater than  17 mm
2  

In [175]:
len( df_joined_hourly[ df_joined_hourly['pluie'] == df_joined_hourly['prcp_bool']  ])

45519

In [184]:
df_joined_hourly[ (df_joined_hourly['pluie'] != df_joined_hourly['prcp_bool']) & (df_joined_hourly['prcp'] > 23)  ]

,idmeteo,idtelescope,tempext,tempint,pression,humidext,humidint,pluie,vent,dirvent,...,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,prcp,prcp_bool
date,,,,,,,,,,,,,,,,,,,,,
2019-06-22 22:00:00,509270.0,3.0,14.3,14.3,1002.533333,92.566667,100.0,0,1.266667,325.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,23.8,1


# No Resampling Method

In [70]:
new_model = df_model

# slice from the first pluie occuring and the last date that's within metar date range
new_model = new_model.iloc[114440:-30200].copy().reset_index(drop=True)

def find_first_gt(element, df):
    all_gts = df[ df['date'].gt( element ) ]
    if len( all_gts ) > 0:
        return all_gts.iloc[0]
    else:
        return np.nan

In [71]:
new_metar = df_metar[ (df_metar['date'].ge( new_model.iloc[0]['date'] )) ].reset_index(drop=True)

In [53]:
def match_metar2model(row, metar, model):
    if row.name > 0:
        model_rows = model['pluie'][ (model['date'].ge( metar.loc[ row.name - 1 ]['date'] )) &
                                   (model['date'].le( row['date'] ) ) 
                                ]
    else: 
        return np.nan
    
    if len( model_rows > 0):
        metar_bool = row['prcp'] > 0
        model_bool = bool( model_rows.max() )
        if metar_bool and model_bool:
            return( "TP" )
        elif metar_bool and not model_bool:
            return( "FN" )
        elif not metar_bool and model_bool:
            return("FP")
        else:
            return("TN")
    else:
        return np.nan


In [75]:
new_metar['agree'] = new_metar.apply( match_metar2model, args=(new_metar,new_model,), axis = 1 )


In [77]:
len(new_metar)

100208

In [174]:
new_metar[ new_metar['agree'] == 'FP' ].iloc[5718] #len = 5719

date     2019-01-19 10:30:00
prcp                       0
agree                     FP
Name: 69874, dtype: object

In [175]:
new_metar.iloc[69864:69884]

,date,prcp,agree
69864,2019-01-19 05:30:00,0.0,TN
69865,2019-01-19 06:00:00,0.0,TN
69866,2019-01-19 06:30:00,0.0,TN
69867,2019-01-19 07:00:00,0.0,TN
69868,2019-01-19 07:30:00,0.0,TN
69869,2019-01-19 08:00:00,0.0,TN
69870,2019-01-19 08:30:00,0.0,TN
69871,2019-01-19 09:00:00,0.0,FP
69872,2019-01-19 09:30:00,0.0,FP
69873,2019-01-19 10:00:00,0.0,FP


In [113]:
model_rows

Series([], Name: pluie, dtype: int64)

In [114]:
new_metar.loc[row.name - 1]

date     2020-11-10 08:55:00
prcp                     0.8
agree                    NaN
Name: 100204, dtype: object

In [115]:
row

date     2020-11-10 09:00:00
prcp                       0
agree                    NaN
Name: 100205, dtype: object

In [109]:
new_model[( new_model['date'].dt.date ==  pd.Timestamp(date(2020, 11, 9))) & 
          ( new_model['date'].ge( new_metar.loc[row.name - 1].date ))][['date','pluie']]

,date,pluie
515240,2020-11-09 16:50:42,0
515241,2020-11-09 17:00:44,0
515242,2020-11-09 17:05:45,0
515243,2020-11-09 17:10:45,0
515244,2020-11-09 17:15:46,0
515245,2020-11-09 17:20:46,0
515246,2020-11-09 17:25:48,0
515247,2020-11-09 17:30:49,0
515248,2020-11-09 17:45:54,0
515249,2020-11-09 17:50:56,0


In [95]:
new_metar[ new_metar['date'].dt.date == pd.Timestamp(date(2020, 11, 9))][['date','prcp']]

,date,prcp
100113,2020-11-09 00:00:00,0.0
100114,2020-11-09 00:30:00,0.0
100115,2020-11-09 01:00:00,0.0
100116,2020-11-09 01:30:00,0.0
100117,2020-11-09 02:00:00,0.0
...,...,...
100174,2020-11-09 22:57:00,0.0
100175,2020-11-09 23:00:00,0.6
100176,2020-11-09 23:04:59,0.4
100177,2020-11-09 23:30:00,0.4


In [61]:
subset.loc[:,('agree')] = subset['agree'].astype('category')

In [69]:
len( subset[ subset['agree'] == "TP" ])

0

In [46]:
match_metar2model( row, new_metar, new_model )

1

In [ ]:
new_metar

In [104]:
new_metar.loc[:,('date')] = pd.to_datetime( new_metar['date'] )
new_model.loc[:,('date')] = pd.to_datetime( new_model['date'])

In [106]:
new_model['pluie'][ new_model['date'].le( new_metar['date'].iloc[5] ) ].max()

1

In [33]:
len(new_model[ new_model['pluie'] > 0 ])/len(new_model)

0.04149326625869446

In [51]:
len(new_model[((new_model['pluie'] > 0) & (new_model['METAR_date'] == 0)) | ((new_model['pluie'] == 0) & (new_model['METAR_date'] > 0))])

33226

In [62]:
print("Rain sensor says rain, METAR says no rain (FP): {0}".format(len(new_model[((new_model['pluie'] > 0) & (new_model['METAR_date'] == 0))])))
print("Rain sensor says no rain, METAR says rain (FN): {0}".format(len(new_model[((new_model['pluie'] == 0) & (new_model['METAR_date'] > 0))])))

Rain sensor says rain, METAR says no rain (FP): 18840
Rain sensor says no rain, METAR says rain (FN): 14386


In [44]:
new_model.iloc[-30200]['date']

Timestamp('2020-07-13 07:39:42')

In [23]:
new_model_check = new_model.iloc[-1000:]
new_model_check[['METAR_prcp','METAR_date']] = new_model_check['date'].apply( find_first_gt, args=(df_metar,) )

KeyError: "None of [Index(['METAR_prcp', 'METAR_date'], dtype='object')] are in the [columns]"

In [ ]:
new_model.to_csv( "C:\\Users\\Matvey Solovyov\\OneDrive\\Documents\\UWA\\2021_S1\\GinginWx\\gingin-weather\\new_joined.csv", index=False)

In [296]:
a1 = new_model['date'].iloc[-1]
a2 = df_metar[ df_metar['date'].gt( a1 ) ]
len(a2)

0

In [4]:
new_model_1 = new_model.iloc[:100001]
new_model_2 = new_model.iloc[100001:200001]
new_model_3 = new_model.iloc[200001:300001]
new_model_4 = new_model.iloc[300001:400001]
new_model_5 = new_model.iloc[400001:500001]
new_model_6 = new_model.iloc[500001:600001]
new_model_7 = new_model.iloc[600001:]

In [5]:
new_model_1[['METAR_prcp','METAR_date']] = new_model_1['date'].apply( find_first_gt, args=(df_metar,) )

C:\Users\Matvey Solovyov\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [304]:
new_model_1[['pluie', 'METAR_prcp']][ (new_model_1['pluie']> 0) | (new_model_1['METAR_prcp'] > 0)  ]

,pluie,METAR_prcp
date,,
2013-11-29 11:50:22,0,0.2
2013-11-29 11:55:23,0,0.2
2013-11-30 13:29:25,0,0.4
2013-11-30 13:34:25,0,0.4
2013-11-30 13:39:26,0,0.4
...,...,...
2014-11-27 13:06:56,0,2.8
2014-11-27 13:11:56,0,2.8
2014-11-27 13:16:57,0,2.8


In [312]:
sub = df_model.iloc[100001:200001]
len(sub[ sub['pluie'] > 0 ])

6582

In [313]:
new_model_2[['METAR_prcp','METAR_date']] = new_model_2['date'].apply( find_first_gt, args=(df_metar,) )
len( new_model_2[ (new_model_2['pluie']> 0) & (new_model_2['METAR_prcp'] > 0)  ] )
# 787 of total 8948 where either is > 0

<ipython-input-313-ba535eda112b>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len( new_model_1[ (new_model_2['pluie']> 0) & (new_model_2['METAR_prcp'] > 0)  ] )


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [317]:
new_model_3[['METAR_prcp','METAR_date']] = new_model_3['date'].apply( find_first_gt, args=(df_metar,) )
print(len( new_model_3[ (new_model_3['pluie']> 0) & (new_model_3['METAR_prcp'] > 0)  ] ))
print(len( new_model_3[ (new_model_3['pluie']> 0) | (new_model_3['METAR_prcp'] > 0)  ] ))
# 1401 / 9798

0
0


<ipython-input-317-957ba24bec03>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(len( new_model_3[ (new_model_2['pluie']> 0) & (new_model_3['METAR_prcp'] > 0)  ] ))
<ipython-input-317-957ba24bec03>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(len( new_model_3[ (new_model_2['pluie']> 0) | (new_model_3['METAR_prcp'] > 0)  ] ))


In [319]:
new_model_4[['METAR_prcp','METAR_date']] = new_model_4['date'].apply( find_first_gt, args=(df_metar,) )
print(len( new_model_4[ (new_model_4['pluie']> 0) & (new_model_4['METAR_prcp'] > 0)  ] ))
print(len( new_model_4[ (new_model_4['pluie']> 0) | (new_model_4['METAR_prcp'] > 0)  ] ))
# 1336 / 10933

<ipython-input-319-5cedf0b0b104>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(len( new_model_4[ (new_model_3['pluie']> 0) & (new_model_4['METAR_prcp'] > 0)  ] ))


0
0


<ipython-input-319-5cedf0b0b104>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(len( new_model_4[ (new_model_3['pluie']> 0) | (new_model_4['METAR_prcp'] > 0)  ] ))


In [321]:
new_model_5[['METAR_prcp','METAR_date']] = new_model_5['date'].apply( find_first_gt, args=(df_metar,) )
print(len( new_model_5[ (new_model_5['pluie']> 0) & (new_model_5['METAR_prcp'] > 0)  ] ))
print(len( new_model_5[ (new_model_5['pluie']> 0) | (new_model_5['METAR_prcp'] > 0)  ] ))
# 1336 / 10933

269
4090


In [322]:
new_model_6[['METAR_prcp','METAR_date']] = new_model_6['date'].apply( find_first_gt, args=(df_metar,) )
print(len( new_model_6[ (new_model_6['pluie']> 0) & (new_model_6['METAR_prcp'] > 0)  ] ))
print(len( new_model_6[ (new_model_6['pluie']> 0) | (new_model_6['METAR_prcp'] > 0)  ] ))
# 1336 / 10933

0
2349


In [323]:
new_model_7[['METAR_prcp','METAR_date']] = new_model_7['date'].apply( find_first_gt, args=(df_metar,) )
print(len( new_model_7[ (new_model_7['pluie']> 0) & (new_model_7['METAR_prcp'] > 0)  ] ))
print(len( new_model_7[ (new_model_7['pluie']> 0) | (new_model_7['METAR_prcp'] > 0)  ] ))
# 1336 / 10933

TypeError: object of type 'float' has no len()